# INDIVIDUAL CLUSTERING

In [1]:
import numpy as np
import datetime
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt

### Detalles

### Lista de Clientes

In [2]:
# Extraemos la lista de clientes sin repetir

def leer_data():
    outfile='./data2/data.csv'
    data = pd.read_csv(outfile)
    return data

data = leer_data()
clientes =  data.groupby('client_id').client_id.count().index

## Temporal TXs footprint

In [3]:
# Definimos la ruta de los archivos 

file='U'
raw_data='./data2/%s.json' %(file)
individual_footprint="%s.individual_footprint" %(raw_data)
individual_clusters="%s.clusters" %(individual_footprint)
individual_labels="%s.labels" %(individual_footprint)

### Funciones

In [4]:
def process_footprint(data,tests,log=False):
    from sklearn.cluster import MiniBatchKMeans
    #KMeans(init='k-means++', n_clusters=k, n_init=10)
    import datetime
    K={}
    for k in tests:
        if k<=len(data):
            if log:
                print("%s: processing %s"%(datetime.datetime.now(),k))
            K[k]=bench_k_means(MiniBatchKMeans(init='k-means++', n_clusters=k, batch_size=100,
                      n_init=10, max_no_improvement=10, verbose=0,
                      random_state=0),name="k-means++", data=data)
    return K

In [5]:
def compute_best_k(x,y,occurrencies, plot=False,points=1000,sf=0.9):
    import numpy as np
    
    if len(x)<5:
        return max(1, round(np.sqrt(occurrencies/2)))
    
    from scipy.interpolate import interp1d
    from scipy.interpolate import UnivariateSpline
    spl = UnivariateSpline(x, y)
    spl.set_smoothing_factor(sf)
    xs = np.linspace(min(x), max(x), points)
    ys = spl(xs)
    idx_better_k=get_change_point(xs, ys)
    if plot:
        import pylab
        pylab.plot(xs,ys)
        
        pylab.scatter(xs[idx_better_k],ys[idx_better_k],s=20, marker='o')
        pylab.text(xs[idx_better_k],ys[idx_better_k],"bestK %s" %(np.round(xs[idx_better_k])))
        return int(np.round(xs[idx_better_k])),pylab
    return int(np.round(xs[idx_better_k]))

In [6]:
def bench_k_means(estimator, name, data,distance_function=None):
    from sklearn import metrics
    from sklearn.metrics import silhouette_samples, silhouette_score
    import time
    t0 = time.time()
    if distance_function:
        estimator.fit(data,distance_function)
    else:
        estimator.fit(data)
    #cluster_labels = estimator.fit_predict(data)
    #silhouette_score_ = silhouette_score(data, cluster_labels)
    
    inertia=estimator.inertia_
    duration=time.time() - t0
    return {'inertia':inertia,'duration':duration, 'estimator':estimator}#,'silhouette':silhouette_score_}

def get_change_point(x, y):
    """
         Elección del mejor K
         :: param x: lista de valores de K
         :: param y: lista de valores de SSE
    """
    import math
    max_d = -float('infinity')
    index = 0

    for i in range(0, len(x)):
        c = closest_point_on_segment(a=[x[0], y[0]], b=[x[len(x)-1], y[len(y)-1]], p=[x[i], y[i]])
        d = math.sqrt((c[0]-x[i])**2 + (c[1]-y[i])**2)
        if d > max_d:
            max_d = d
            index = i
    
    return index

def closest_point_on_segment(a, b, p):
    sx1 = a[0]
    sx2 = b[0]
    sy1 = a[1]
    sy2 = b[1]
    px = p[0]
    py = p[1]

    x_delta = sx2 - sx1
    y_delta = sy2 - sy1

    if x_delta == 0 and y_delta == 0:
        return p

    u = ((px - sx1) * x_delta + (py - sy1) * y_delta) / (x_delta * x_delta + y_delta *  y_delta)
    if u < 0:
        closest_point = a
    elif u > 1:
        closest_point = b
    else:
        cp_x = sx1 + u * x_delta
        cp_y = sy1 + u * y_delta
        closest_point = [cp_x, cp_y]

    return closest_point
	

### Individual Clustering

In [7]:
# Numero de filas del archivo
f=open(individual_footprint)
num_rows = len(f.readlines())-1
f.close()

#<customer_id;year;week;profile_id;size;t1... tn >
import datetime
f=open(individual_footprint)
fw=open(individual_clusters,'w')  #uid,cluster_id,centroid
fw2=open(individual_labels,'w') #uid,year,week,cluster_id,profile
fw.write('customer_tag;individual_cluster;d0t0;d1t0;d2t0;d3t0;d4t0;d5t0;d6t0;d0t1;d1t1;d2t1;d3t1;d4t1;d5t1;d6t1;d0t2;d1t2;d2t2;d3t2;d4t2;d5t2;d6t2;d0t3;d1t3;d2t3;d3t3;d4t3;d5t3;d6t3\n')
fw2.write('customer_tag;year;week;individual_cluster;d0t0;d1t0;d2t0;d3t0;d4t0;d5t0;d6t0;d0t1;d1t1;d2t1;d3t1;d4t1;d5t1;d6t1;d0t2;d1t2;d2t2;d3t2;d4t2;d5t2;d6t2;d0t3;d1t3;d2t3;d3t3;d4t3;d5t3;d6t3\n')

f.readline()
data=[] #buffer

footprints_clustered=0
footprints_clusters=0
n_cliente=0
contador = 0
temporal= 0
for row in f: #reading individual footprint
    row=row.strip().split(',') # leemos cada elemento da linea parseada por ","
    uid=row[0]
    year=row[1]
    week=row[2]
    size=int(row[4])
    profile=np.array([float(el) for el in row[5:]])
    # Individual clustering
    if uid==clientes[n_cliente]: # Para cada fila donde los "uid" son iguales 
        data.append(((uid,year,week),profile))     
        contador+=1
    else: #final de cliente
        
        #---------------------------------------------------------------------
        # procesar data
        #---------------------------------------------------------------------
        to_cluster=[el[1] for el in data]
        K=process_footprint(to_cluster,np.arange(1,len(to_cluster)+1))

        # choose k
        x=list(K.keys())
        y=[K[k]['inertia'] for k in K]
        best_k=compute_best_k(x,y,len(to_cluster))
        print(str(contador)+' => clustering: '+str(clientes[n_cliente])+' len data: '+str(len(data))+" best k: "+str(best_k))
        
        # clustering
        if best_k==1:
            #to few records
            cluster_centers_=[np.average(to_cluster,axis=0)]
            labels_=[0]*len(to_cluster)  
        else:
            cluster_centers_=K[best_k]['estimator'].cluster_centers_
            labels_=K[best_k]['estimator'].labels_
        
        #export individual centroids
        for i in np.arange(len(cluster_centers_)):
            string="%s;%s;%s\n"%(clientes[n_cliente],i,';'.join([str(el) for el in cluster_centers_[i]])) #uid,cluster_id,centroid
            fw.write(string)
            footprints_clusters+=1
        fw.flush()

        #export original data and labels
        for i in np.arange(len(data)):
            uid2=data[i][0]
            profile2=data[i][1]
            label2=labels_[i]
            string="%s;%s;%s;%s;%s\n" %(uid2[0],uid2[1],uid2[2],label2
                                                    ,';'.join([str(el) for el in profile2]))#uid,year,week,cluster_id,profile
            fw2.write(string)
            footprints_clustered+=1
        fw2.flush()
        #---------------------------------------------------------------------
        #---------------------------------------------------------------------
        
        

        data=[] #buffer
        data.append(((uid,year,week),profile))
        
        contador+=1
        temporal+=1
        n_cliente+=1
        
    if contador == num_rows:        # Para el ultimo cliente y ultima fila
        #---------------------------------------------------------------------
        # procesar data
        #---------------------------------------------------------------------
        to_cluster=[el[1] for el in data]
        K=process_footprint(to_cluster,np.arange(1,len(to_cluster)+1))

        # choose k
        x=list(K.keys())
        y=[K[k]['inertia'] for k in K]
        best_k=compute_best_k(x,y,len(to_cluster))
        print(str(contador)+' => clustering: '+str(clientes[n_cliente])+' len data: '+str(len(data))+" best k: "+str(best_k))
        
        # clustering
        if best_k==1:
            #to few records
            cluster_centers_=[np.average(to_cluster,axis=0)]
            labels_=[0]*len(to_cluster)  
        else:
            cluster_centers_=K[best_k]['estimator'].cluster_centers_
            labels_=K[best_k]['estimator'].labels_
        
        #export individual centroids
        for i in np.arange(len(cluster_centers_)):
            string="%s;%s;%s\n"%(uid,i,';'.join([str(el) for el in cluster_centers_[i]])) #uid,cluster_id,centroid
            fw.write(string)
            footprints_clusters+=1
        fw.flush()

        #export original data and labels
        for i in np.arange(len(data)):
            uid2=data[i][0]
            profile2=data[i][1]
            label2=labels_[i]
            string="%s;%s;%s;%s;%s\n" %(uid2[0],uid2[1],uid2[2],label2
                                                    ,';'.join([str(el) for el in profile2]))#uid,year,week,cluster_id,profile
            fw2.write(string)
            footprints_clustered+=1
        fw2.flush()
        #---------------------------------------------------------------------
        #---------------------------------------------------------------------
        
        print("final")   
    
    

temporal

1 => clustering: +++g8j9k+5A= len data: 1 best k: 1
2 => clustering: ++/oQ9Lb9dI= len data: 1 best k: 1
5 => clustering: ++14g8obpj0= len data: 3 best k: 1
6 => clustering: ++1XMtcwMec= len data: 1 best k: 1
7 => clustering: ++3gxZFOJCM= len data: 1 best k: 1
8 => clustering: ++438ugzEhg= len data: 1 best k: 1
9 => clustering: ++5u+heOZ8o= len data: 1 best k: 1
13 => clustering: ++70ByX0a3Q= len data: 4 best k: 1
14 => clustering: ++7i5fi6kBU= len data: 1 best k: 1
16 => clustering: ++834mPfd7g= len data: 2 best k: 1
17 => clustering: ++8H96p+hLk= len data: 1 best k: 1
23 => clustering: ++93ZWJ5ONQ= len data: 6 best k: 4
24 => clustering: ++9tB+M5oWk= len data: 1 best k: 1
25 => clustering: ++C9HYE9YAk= len data: 1 best k: 1
26 => clustering: ++FMZ01PEY4= len data: 1 best k: 1
27 => clustering: ++FVgfEREuc= len data: 1 best k: 1
28 => clustering: ++JHadWy+ns= len data: 1 best k: 1
29 => clustering: ++JxWznHvfc= len data: 1 best k: 1
30 => clustering: ++KZ8A+f7MY= len data: 1 best k: 1


421 => clustering: +0uLvWNH8ic= len data: 9 best k: 4
422 => clustering: +0ugFUoLcZw= len data: 1 best k: 1
424 => clustering: +0vMG+809FA= len data: 2 best k: 1
425 => clustering: +0wZhwzyzJo= len data: 1 best k: 1
426 => clustering: +0xKwvsIkFA= len data: 1 best k: 1
427 => clustering: +0xr3WGLoS4= len data: 1 best k: 1
428 => clustering: +0yTOXnCLJI= len data: 1 best k: 1
429 => clustering: +0zRdwVpJz4= len data: 1 best k: 1
431 => clustering: +13/Y57RH00= len data: 2 best k: 1
433 => clustering: +15MZjEFNN4= len data: 2 best k: 1
434 => clustering: +15spL7OWvQ= len data: 1 best k: 1
440 => clustering: +1670k6aThA= len data: 6 best k: 3
441 => clustering: +182ID1sv5I= len data: 1 best k: 1
442 => clustering: +19b5bgGBMQ= len data: 1 best k: 1
448 => clustering: +1A+siRRE78= len data: 6 best k: 3
450 => clustering: +1D4fLSNElU= len data: 2 best k: 1
455 => clustering: +1DMXF3J0z8= len data: 5 best k: 3
458 => clustering: +1IQZkOp930= len data: 3 best k: 1
468 => clustering: +1JI7hdtS

872 => clustering: +4+ccMb2CoA= len data: 12 best k: 5
873 => clustering: +4/ElXyX6GI= len data: 1 best k: 1
875 => clustering: +47PkrKmzrM= len data: 2 best k: 1
876 => clustering: +47zbC+XIp8= len data: 1 best k: 1
877 => clustering: +4Co4wNMRs0= len data: 1 best k: 1
878 => clustering: +4DAjBvBsLY= len data: 1 best k: 1
879 => clustering: +4F9gT4gLAY= len data: 1 best k: 1
917 => clustering: +4F9nGxb1CY= len data: 38 best k: 10
918 => clustering: +4FII2KjFAI= len data: 1 best k: 1
919 => clustering: +4FMUyKjKWk= len data: 1 best k: 1
921 => clustering: +4FZhZJwt+Y= len data: 2 best k: 1
924 => clustering: +4J1a/e0lBA= len data: 3 best k: 1
927 => clustering: +4JY45X1ipg= len data: 3 best k: 1
929 => clustering: +4JlzJ/3YM8= len data: 2 best k: 1
931 => clustering: +4LHdQSxUeA= len data: 2 best k: 1
932 => clustering: +4LNqs4z/ME= len data: 1 best k: 1
934 => clustering: +4MHYTV5NhE= len data: 2 best k: 1
935 => clustering: +4MYsaW+zZU= len data: 1 best k: 1
937 => clustering: +4MjBW

1318 => clustering: +70+Hc1nrw0= len data: 16 best k: 6
1320 => clustering: +735pM7q6pg= len data: 2 best k: 1
1321 => clustering: +73Rfc3uF+4= len data: 1 best k: 1
1326 => clustering: +75MTWLveq8= len data: 5 best k: 3
1327 => clustering: +774MXQOkJc= len data: 1 best k: 1
1328 => clustering: +79fFDYvcmk= len data: 1 best k: 1
1330 => clustering: +79wHafD5Wo= len data: 2 best k: 1
1332 => clustering: +79ytbkLUNA= len data: 2 best k: 1
1333 => clustering: +7AY03ecTdA= len data: 1 best k: 1
1334 => clustering: +7Bjc0s276k= len data: 1 best k: 1
1336 => clustering: +7BrjCQ3gLI= len data: 2 best k: 1
1338 => clustering: +7CEcPAHU1U= len data: 2 best k: 1
1340 => clustering: +7DqKy2zRnk= len data: 2 best k: 1
1343 => clustering: +7EH918Dw5g= len data: 3 best k: 1
1344 => clustering: +7HxRj9wf1M= len data: 1 best k: 1
1345 => clustering: +7IUDgwIFnI= len data: 1 best k: 1
1346 => clustering: +7K5/V2MOFs= len data: 1 best k: 1
1349 => clustering: +7L5vnbEdlQ= len data: 3 best k: 1
1351 => c

1680 => clustering: +9syP8mKxBg= len data: 17 best k: 3
1681 => clustering: +9tbT3X+IYM= len data: 1 best k: 1
1682 => clustering: +9vw3JX+nQ4= len data: 1 best k: 1
1692 => clustering: +9wNyZkq684= len data: 10 best k: 6
1700 => clustering: +9x6WLEWIDI= len data: 8 best k: 5
1701 => clustering: +9yJoBsAES4= len data: 1 best k: 1
1702 => clustering: +9zI/m/hvVY= len data: 1 best k: 1
1706 => clustering: +9zM3Smnd+0= len data: 4 best k: 1
1708 => clustering: +9zkYu2Y5i8= len data: 2 best k: 1
1709 => clustering: +A+1O8b2RcM= len data: 1 best k: 1
1712 => clustering: +A0zWFEVbSA= len data: 3 best k: 1
1717 => clustering: +A1r7EOnPiM= len data: 5 best k: 2
1720 => clustering: +A3Lp8W8aFY= len data: 3 best k: 1
1722 => clustering: +A5q+QqzcCY= len data: 2 best k: 1
1723 => clustering: +A6M5V8hATw= len data: 1 best k: 1
1725 => clustering: +A8jnXMhYfc= len data: 2 best k: 1
1729 => clustering: +AAODo1Q4FE= len data: 4 best k: 1
1733 => clustering: +ACXKO05Ewc= len data: 4 best k: 1
1734 => 

...

17672 => clustering: /yozXqKlDP0= len data: 21 best k: 7
17674 => clustering: /yr+zPMeQ4w= len data: 2 best k: 1
17677 => clustering: /yvhvz02/q0= len data: 3 best k: 1
17679 => clustering: /ywfTu0pcGw= len data: 2 best k: 1
17682 => clustering: /yxp4/aFyDs= len data: 3 best k: 1
17683 => clustering: /yy9ETqm0fg= len data: 1 best k: 1
17685 => clustering: /yz6PifRlPw= len data: 2 best k: 1
17686 => clustering: /z+XOO2RdUU= len data: 1 best k: 1
17687 => clustering: /z0YXD3Zvjo= len data: 1 best k: 1
17688 => clustering: /z1S0A667vM= len data: 1 best k: 1
17689 => clustering: /z1amB8om7Q= len data: 1 best k: 1
17690 => clustering: /z2tEYH2QC8= len data: 1 best k: 1
17691 => clustering: /z3YmluxpzI= len data: 1 best k: 1
17692 => clustering: /z3c63zj8jQ= len data: 1 best k: 1
17693 => clustering: /z4vM54DSJE= len data: 1 best k: 1
17694 => clustering: /z5t7ONf7fc= len data: 1 best k: 1
17700 => clustering: /z6154fiuXc= len data: 6 best k: 3
17701 => clustering: /z9vcWnD8Ww= len data: 1 b

18090 => clustering: 0/rnnQElNEc= len data: 1 best k: 1
18091 => clustering: 0/tv2M25Lx8= len data: 1 best k: 1
18092 => clustering: 0/u2iYu3R10= len data: 1 best k: 1
18101 => clustering: 0/uXymiWa0Y= len data: 9 best k: 4
18103 => clustering: 0/vtsAGe2CU= len data: 2 best k: 1
18105 => clustering: 0/wXCdpT1+Y= len data: 2 best k: 1
18106 => clustering: 0/x8XXI2Uhg= len data: 1 best k: 1
18110 => clustering: 0/y8Zyy/sGg= len data: 4 best k: 1
18111 => clustering: 0/yc3NheVP0= len data: 1 best k: 1
18115 => clustering: 0/yrnt3NxVQ= len data: 4 best k: 1
18119 => clustering: 0/zjEmeVfKA= len data: 4 best k: 1
18122 => clustering: 0/zsTuSMkvQ= len data: 3 best k: 1
18123 => clustering: 00+Fmn09kpI= len data: 1 best k: 1
18127 => clustering: 00+KIAjXNko= len data: 4 best k: 1
18128 => clustering: 000aiC7bp5w= len data: 1 best k: 1
18130 => clustering: 001IyiSRfZw= len data: 2 best k: 1
18131 => clustering: 001SH2TWf8A= len data: 1 best k: 1
18133 => clustering: 003sINYwIJk= len data: 2 be

18432 => clustering: 02CChpqmUD8= len data: 1 best k: 1
18434 => clustering: 02Ci86IFOro= len data: 2 best k: 1
18440 => clustering: 02E2wOYneZA= len data: 6 best k: 3
18441 => clustering: 02EjHUBjQq8= len data: 1 best k: 1
18442 => clustering: 02F6edloHc4= len data: 1 best k: 1
18443 => clustering: 02FOrMY46f4= len data: 1 best k: 1
18445 => clustering: 02FoeTmqKFk= len data: 2 best k: 1
18446 => clustering: 02KUFbYfP/g= len data: 1 best k: 1
18449 => clustering: 02KvxPY+K3k= len data: 3 best k: 1
18452 => clustering: 02O5yoSR7T4= len data: 3 best k: 1
18453 => clustering: 02P1S6AUl48= len data: 1 best k: 1
18456 => clustering: 02QP91Tf9Wc= len data: 3 best k: 1
18457 => clustering: 02QYaLq2Rws= len data: 1 best k: 1
18458 => clustering: 02SQJkTNhFI= len data: 1 best k: 1
18459 => clustering: 02SlD5xAIiI= len data: 1 best k: 1
18460 => clustering: 02TW3G1xqyo= len data: 1 best k: 1
18461 => clustering: 02VkKKAbHqM= len data: 1 best k: 1
18462 => clustering: 02WZ5ar0Zss= len data: 1 be

18806 => clustering: 04nwCxcXf8c= len data: 1 best k: 1
18807 => clustering: 04pcmsWu6pU= len data: 1 best k: 1
18808 => clustering: 04px9D88g8s= len data: 1 best k: 1
18809 => clustering: 04pxRmnIrVI= len data: 1 best k: 1
18810 => clustering: 04qJilb2y/M= len data: 1 best k: 1
18811 => clustering: 04uYdrzLhto= len data: 1 best k: 1
18812 => clustering: 04v9kUL+mTA= len data: 1 best k: 1
18813 => clustering: 04xY7CIp1dE= len data: 1 best k: 1
18814 => clustering: 04y4i/ijolI= len data: 1 best k: 1
18815 => clustering: 04ycG1CkwTE= len data: 1 best k: 1
18822 => clustering: 04yn0i1Eh1w= len data: 7 best k: 3
18823 => clustering: 04ziQAcllDs= len data: 1 best k: 1
18826 => clustering: 05/qZu3kV5g= len data: 3 best k: 1
18829 => clustering: 0502a0cyKkc= len data: 3 best k: 1
18832 => clustering: 050KU71+jss= len data: 3 best k: 1
18835 => clustering: 050d7NugD7c= len data: 3 best k: 1
18836 => clustering: 051FwLTkDJQ= len data: 1 best k: 1
18842 => clustering: 054oIelYmJs= len data: 6 be

7584